In [48]:
import pandas as pd
import numpy as np
data = pd.read_csv("./mobike_shanghai_sample_updated.csv")
data.head(10)

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1..."
1,891333,92776,6605,2016-08-29 19:09,121.508,31.279,2016-08-29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1..."
2,1106623,152045,8876,2016-08-13 16:17,121.383,31.254,2016-08-13 16:36,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1..."
3,1389484,196259,10648,2016-08-23 21:34,121.484,31.320,2016-08-23 21:43,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1..."
4,188537,78208,11735,2016-08-16 07:32,121.407,31.292,2016-08-16 07:41,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1..."
5,537030,66346,10335,2016-08-07 21:00,121.488,31.224,2016-08-07 21:20,121.468,31.233,"121.468,31.233#121.469,31.233#121.470,31.233#1..."
6,517706,99631,11258,2016-08-29 13:39,121.445,31.176,2016-08-29 13:54,121.447,31.192,"121.445,31.176#121.446,31.175#121.446,31.176#1..."
7,270836,63136,11361,2016-08-29 09:21,121.369,31.252,2016-08-29 13:43,121.376,31.233,"121.367,31.238#121.367,31.239#121.368,31.240#1..."
8,441362,331921,11705,2016-08-31 08:16,121.529,31.324,2016-08-31 08:19,121.532,31.323,"121.529,31.324#121.530,31.324#121.531,31.323#1..."
9,76435,347335,8135,2016-08-30 12:49,121.467,31.320,2016-08-30 13:03,121.447,31.318,"121.447,31.317#121.447,31.318#121.448,31.314#1..."


# 聚类数量为10（候选点数量为5，选3个）

In [143]:
cluster_num = 1000
wait_list_num = 500
p = 200
from sklearn.cluster import KMeans
km = KMeans(n_clusters = cluster_num)
X = data.loc[:,["start_location_x","start_location_y"]]
print(X)

        start_location_x  start_location_y
0                121.348            31.389
1                121.508            31.279
2                121.383            31.254
3                121.484            31.320
4                121.407            31.292
...                  ...               ...
102356           121.478            31.297
102357           121.320            31.238
102358           121.391            31.307
102359           121.515            31.269
102360           121.521            31.261

[102361 rows x 2 columns]


In [144]:
km.fit(X)
centers = km.cluster_centers_
centers = np.array(centers)
print(centers)
centers = centers[:,0:2]
print(centers)

[[121.48451064  31.23020213]
 [121.36782558  31.2085814 ]
 [121.48289276  31.29002145]
 ...
 [121.5385      31.044625  ]
 [121.40985542  31.22693976]
 [121.2625      31.15525   ]]
[[121.48451064  31.23020213]
 [121.36782558  31.2085814 ]
 [121.48289276  31.29002145]
 ...
 [121.5385      31.044625  ]
 [121.40985542  31.22693976]
 [121.2625      31.15525   ]]


In [145]:
centers[1]

array([121.36782558,  31.2085814 ])

In [146]:
#定义曼哈顿距离函数
def MD(vector1, vector2):
    d = np.linalg.norm(vector1-vector2, ord=1)
    return d

In [147]:
#计算各点需求量和候选点与需求点两两间的距离
label = km.labels_
X["label"] = label
#X["label"].value_counts()
import numpy as np
d_set = np.zeros(cluster_num)
for item in label:
    d_set[item] = d_set[item] + 1
d_set
import heapq    
top_index_list = heapq.nlargest(wait_list_num, range(len(d_set)), d_set.take)
print(top_index_list)
c_set = np.zeros((cluster_num, wait_list_num))
for i in range(cluster_num):
    for j in range(len(top_index_list)):
        c_set[i][j] = MD(centers[i], centers[top_index_list[j]])
c_set

[687, 278, 2, 400, 213, 105, 3, 382, 430, 954, 396, 547, 313, 392, 855, 5, 289, 541, 157, 468, 101, 586, 282, 493, 627, 104, 592, 781, 516, 241, 355, 918, 117, 178, 108, 37, 394, 775, 19, 564, 175, 66, 525, 551, 16, 268, 490, 816, 67, 133, 302, 836, 272, 920, 373, 847, 153, 154, 349, 821, 155, 416, 426, 730, 84, 417, 470, 676, 320, 545, 187, 161, 322, 563, 859, 143, 274, 288, 333, 789, 894, 78, 103, 148, 212, 398, 484, 95, 51, 263, 747, 522, 346, 572, 218, 967, 628, 688, 710, 47, 331, 348, 753, 463, 870, 874, 379, 732, 951, 195, 232, 111, 225, 606, 925, 542, 482, 713, 880, 271, 295, 401, 342, 673, 756, 306, 593, 168, 410, 449, 555, 558, 721, 946, 252, 316, 839, 961, 317, 538, 621, 761, 167, 464, 477, 604, 321, 421, 774, 1, 23, 928, 523, 549, 651, 799, 22, 203, 304, 927, 299, 677, 728, 221, 514, 680, 9, 52, 57, 99, 740, 15, 176, 216, 734, 887, 150, 255, 467, 96, 163, 347, 70, 488, 727, 89, 110, 580, 896, 408, 714, 831, 211, 291, 404, 436, 34, 55, 76, 466, 609, 788, 156, 784, 284, 664, 4

array([[0.1401617 , 0.0904618 , 0.0614372 , ..., 0.20946013, 0.11467693,
        0.        ],
       [0.27846749, 0.22707478, 0.19650723, ..., 0.07115435, 0.06419302,
        0.13830579],
       [0.08196026, 0.03056754, 0.        , ..., 0.26766158, 0.13231421,
        0.0614372 ],
       ...,
       [0.28526011, 0.33002828, 0.30100369, ..., 0.31839079, 0.35424342,
        0.23956649],
       [0.21807929, 0.16668657, 0.13611903, ..., 0.13154255, 0.04328408,
        0.07791759],
       [0.43712447, 0.38573175, 0.35516421, ..., 0.08750263, 0.22285   ,
        0.29696277]])

In [148]:
#将d_i和c_ij乘在一起构成目标函数中的系数
coeffcient_matrix = np.zeros((len(c_set), len(c_set[0])))
for i in range(len(c_set)):
    for j in range(len(c_set[0])):
        coeffcient_matrix[i][j] = d_set[i]*c_set[i][j]
print(coeffcient_matrix)

[[13.1752      8.50340876  5.77509651 ... 19.68925263 10.77963158
   0.        ]
 [47.89640851 39.05686131 33.79924397 ... 12.23854737 11.0412
  23.78859574]
 [30.5711766  11.40169343  0.         ... 99.83776842 49.3532
  22.91607447]
 ...
 [ 2.28208085  2.64022628  2.40802949 ...  2.54712632  2.83394737
   1.91653191]
 [18.10058085 13.8349854  11.29787936 ... 10.91803158  3.59257895
   6.46715957]
 [ 1.74849787  1.54292701  1.42065684 ...  0.35001053  0.8914
   1.18785106]]


## p-中值模型

In [149]:
#调用Gurobi求解
import gurobipy as grb
from gurobipy import GRB
import time
#创建一个名字为p_median_problem的模型
start = time.time()
m = grb.Model("p_median_problem")

#添加决策变量
y = m.addVars(len(c_set), len(c_set[0]), vtype = grb.GRB.BINARY, name = "y")
x = m.addVars(len(c_set[0]), vtype = grb.GRB.BINARY, name = "x")

#添加约束
#y矩阵按行求和值为1
m.addConstrs((y.sum(i,"*") == 1 for i in range(len(c_set))), name = "demand") 
#x向量各分量求和等于p,只添加一个约束函数名不能加s!addConstr
m.addConstr((x.sum("*") == p), name = "pvalue") 

for i in range(len(c_set)):
    for j in range(len(c_set[0])):
        m.addConstr((y[i,j] <= x[j]), name="ouhe")

#添加目标函数
c1 = []
for i in range(len(c_set)):
    for j in range(len(c_set[0])):
        c1.append((i,j))
coeff = grb.tupledict(c1)
for i in range(len(c_set)):
    for j in range(len(c_set[0])):
        coeff[(i,j)] = coeffcient_matrix[i][j]
obj = y.prod(coeff)
m.setObjective(obj, GRB.MINIMIZE)

#求解模型
m.optimize()
time_used = time.time() - start
print("Gurobi求解时间为：" + str(time_used) + "秒")
#输出结果
# for k in range(len(m.getVars())):
#     print('%s %g' % (m.getVars()[k].varName, m.getVars()[k].x), end=" ")
#     if (k+1) % 5 == 0:
#         print()


Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 501001 rows, 500500 columns and 1500500 nonzeros
Model fingerprint: 0xdd7e23ad
Variable types: 0 continuous, 500500 integer (500500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve time: 4.72s
Presolved: 501001 rows, 500500 columns, 1500500 nonzeros
Variable types: 0 continuous, 500500 integer (500500 binary)
Found heuristic solution: objective 1075.3315328

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1710087e+03   0.000000e+00   8.785314e+05      9s
    3102    9.9633888e+02   0.000000e+00   1.077275e+03     11s
    5310    8.6226754e+02   0.000000e+00   6.082486e+0

### 次梯度算法代码主流程参考https://zhuanlan.zhihu.com/p/145944142

In [150]:
#函数封装
def solve_subproblem_1(c_set, d_set, lamda_set):
    import copy
    import numpy as np
    sol_index_list = []
    sol_value_list = []
    A = copy.deepcopy(c_set)
    for i in range(len(A)):
        for j in range(len(A[0])):
            A[i][j] = A[i][j]*d_set[i] + lamda_set[i][j]
    A_matrix = np.array(A)
    #找到A矩阵每行的最小值对应的列索引
    for i in range(len(A)):
        sol_index_list.append(np.argmin(A[i]))
        sol_value_list.append(np.min(A[i]))
    sol_value = np.array(sol_value_list).sum()
    sol_matrix = np.zeros((len(A), len(A[0])))
    for i in range(len(A)):
        sol_matrix[i][sol_index_list[i]]=1
    return sol_matrix,sol_value

def solve_subproblem_2(p, lamda_set):
    import numpy as np
    import heapq
    #lamda_set类型可构造为矩阵传入
    sum_by_col = lamda_set.sum(axis=0)
    top_p_index_list = heapq.nlargest(p,range(len(sum_by_col)),sum_by_col.take)
    sol_vector = np.zeros(len(lamda_set[0]))
    sol_value = 0
    for index in top_p_index_list:
        sol_vector[index] = 1
        sol_value = sol_value + sum_by_col[index]
    return sol_vector, -sol_value

def get_subgradient(y_matrix, x_vector):
    import numpy as np
    s_matrix = np.zeros((len(y_matrix), len(y_matrix[0])))
    for i in range(len(s_matrix)):
        for j in range(len(s_matrix[0])):
            s_matrix[i][j] = y_matrix[i][j] - x_vector[j]
    #将次梯度以矩阵形式存储，事实上其是一个向量
    return s_matrix

def update_subgradient(lamda_set, s_matrix, theta):
    tag = 1 #标记次梯度是否为零
    if np.all(s_matrix == 0):
        tag = 0
        lamda_set_next = np.zeros(len(lamda_set), len(lamda_set[0]))
    else:
        lamda_set_next = lamda_set + theta*s_matrix
        if np.all(lamda_set_next >= 0):
            pass
        else:
            lamda_set_next = np.zeros((len(lamda_set), len(lamda_set[0])))
    return tag, lamda_set_next

In [151]:
#判断次梯度算法得到的解是否为可行解
def is_feasible_sol(x_vector, y_matrix):
    infeasible_loc = []
    for i in range(len(y_matrix)):
        for j in range(len(y_matrix[i])):
            if y_matrix[i][j] > x_vector[j]:
                infeasible_loc.append((i,j))
    if len(infeasible_loc) == 0:
        return infeasible_loc, True
    else:
        return infeasible_loc, False

In [152]:
#将得到的解可行化
def make_it_feasible(c_set, sol_x_vector, sol_y_matrix, infeasible_loc):
    import copy 
    for item in infeasible_loc:
        feasible_list = []
        dic = {}
        for i in range(len(sol_x_vector)):
            if sol_x_vector[i] == 1:
                dic["no"] = i
                dic["c_value"] = c_set[item[0]][i]
                dic_1 = copy.deepcopy(dic)
                feasible_list.append(dic_1)
        min_index = feasible_list[0]["no"]
        min_value = feasible_list[0]["c_value"]
        #print("初始min_index:" + str(min_index))
        #print("初始min_value:" + str(min_value))
        for item_1 in feasible_list:
            c = item_1["c_value"]
            #print("对应c矩阵中的值为:" + str(c))
            if item_1["c_value"] < min_value:
                min_value = item_1["c_value"]
                min_index = item_1["no"]
        #print("改变后min_index:" + str(min_index))
        sol_y_matrix[item[0]][item[1]] = 0
        sol_y_matrix[item[0]][min_index] = 1
#计算目标函数值
def get_object_value(sol_y_matrix, c_set, d_set):
    sum = 0
    for i in range(len(c_set)):
        for j in range(len(c_set[0])):
            sum = sum + d_set[i]*c_set[i][j]*sol_y_matrix[i][j]
    return sum

In [154]:
#在算法停止准则的选用中，选择最简单的控制迭代次数
import numpy as np
import math
import copy
#初始化
import time 
start_1 = time.time()
max_itertimes = 100
problem_size = (cluster_num, wait_list_num)
lamda_set = 4*np.ones(problem_size)
theta = 2
rou = 0.9
z_LR_list = []
best_feasible_val = 0
for i in range(max_itertimes):
    sol_y_matrix, z_LR1 = solve_subproblem_1(c_set, d_set, lamda_set)
    sol_x_vector, z_LR2 = solve_subproblem_2(p, lamda_set)
    z_LR = z_LR1 + z_LR2
    subgradient_matrix = get_subgradient(sol_y_matrix, sol_x_vector)
#     print("本次迭代次梯度为：")
#     print(subgradient_matrix)
    theta = theta*(rou**i)
    tag,lamda_set = update_subgradient(lamda_set, subgradient_matrix, theta)
#     print("更新后lamda值为：")
#     print(lamda_set)
    if tag == 0:
        #print("次梯度为零向量，达到最优解")
        break
    z_LR_list.append(z_LR)
    #print("第" + str(i) + "次迭代目标函数值为：" + str(z_LR))
    if i >= 5:
        if z_LR_list[i] == z_LR_list[i-1] and z_LR_list[i] == z_LR_list[i-2]:
            #print("连续三次值未改变，停止迭代，对偶问题值变化如下")
#             import matplotlib.pyplot as plt
#             plt.plot(np.arange(1, len(z_LR_list)+1), z_LR_list)
#             plt.show()
            break
tag_sol_feasible = True
tag_sol_feasible_new = False
infeasible_loc = []
infeasible_loc, tag_sol_feasible = is_feasible_sol(sol_x_vector, sol_y_matrix)
if tag_sol_feasible == True:
    best_feasible_val = get_object_value(sol_y_matrix, c_set, d_set)
    best_feasible_sol_x = copy.deepcopy(sol_x_vector)
    best_feasible_sol_y = copy.deepcopy(sol_y_matrix)
else:
    #print("违背原约束的y_ij位置为：")
    #print(infeasible_loc)
    make_it_feasible(c_set, sol_x_vector, sol_y_matrix, infeasible_loc)
    infeasible_loc, tag_sol_feasible_new=is_feasible_sol(sol_x_vector, sol_y_matrix)
    #if tag_sol_feasible_new == True:
        #print("已修正为可行解")
    best_feasible_sol_x = copy.deepcopy(sol_x_vector)
    best_feasible_sol_y = copy.deepcopy(sol_y_matrix)
    best_feasible_val = get_object_value(sol_y_matrix, c_set, d_set)
time_used = time.time() - start_1
print("经启发式修正后可行的求解结果为：")
#print("松弛后问题目标函数值z_LR为：" + str(best_feasible_val))
# print("x的解为：")
# print(best_feasible_sol_x)
# print("y的解为：")
# print(best_feasible_sol_y)
print("目标函数值为：")
print(best_feasible_val)
print("拉格朗日启发式算法求解时间为：" + str(time_used) + "秒")

经启发式修正后可行的求解结果为：
目标函数值为：
1626.43733450344
拉格朗日启发式算法求解时间为：40.247262716293335秒
